In [32]:
from azureml.core import Workspace, Experiment, Run,Datastore, Dataset,Environment,Model
from azureml.core.compute import AksCompute, ComputeTarget
#import condadependencies
from azureml.core.environment import CondaDependencies
import joblib
from azureml.core.model import InferenceConfig
#import akswebservice
import pandas as pd 
from azureml.core.webservice import AksWebservice

In [18]:
ws=Workspace.from_config("C:/Users/satya/Downloads/Azure/config.json")

In [19]:
#list datasets
for ds in ws.datasets:
    print(ds)

In [10]:
exp=Environment(name='myexp')
expdep=CondaDependencies.create(pip_packages=['azureml-defaults'],conda_packages=['scikit-learn', 'pandas', 'pip'])
exp.python.conda_dependencies=expdep
exp.register(ws)

{
    "assetId": "azureml://locations/eastus2/workspaces/ba6f060c-e5fc-41d1-b232-49c78f2ca886/environments/myexp/versions/1",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220708.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "myexp",
    "python": {
       

In [14]:
#print registered datastores
for ds in ws.datastores:
    print(ds)
for ds in ws.datasets:
    print(ds)

adultincomedatastore
adultds2
workspaceworkingdirectory
workspaceartifactstore
workspaceblobstore
workspacefilestore
TabularDataset
{
  "source": [
    "('adultds2', 'adultincome+trunc.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}


In [20]:
datastore_access=Datastore.register_azure_blob_container(ws,datastore_name='adultincomedatastore',container_name='adults',account_key='V+oQHRwAvV7IzXJQN2AaKk25cxHvTbtzmQSA/Pl1WY3jRb5PmNmn2/UjlWpQADclivBV3ylUfn/r+ASt/xmtrw=='
,account_name='adultincomestor01')
datastore=Datastore.get(ws,'adultincomedatastore')



In [22]:
path=[(datastore,'adultincome+trunc.csv')]

dataset=Dataset.Tabular.from_delimited_files(path=path)
dataset=dataset.register(ws,'adultdataset',create_new_version=True)

In [26]:
dataset=dataset.to_pandas_dataframe()

In [28]:
#creating the joblib dumps for model registration and deployment
experiment_deploymeny=Experiment(ws,'experiment_deploymeny')
run=experiment_deploymeny.start_logging()

x=dataset.iloc[:,:-1]
y=dataset.iloc[:,-1]
x=pd.get_dummies(x)
train_encoded_cols=x.columns
y=pd.get_dummies(y)
y=y.iloc[:,-1]
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
rfc=RandomForestClassifier()
trained_model=rfc.fit(x_train,y_train)
y_predict=rfc.predict(x_test)
y_prob=rfc.predict_proba(x_test)[:,1]
from sklearn.metrics import accuracy_score,roc_auc_score, confusion_matrix
cm=confusion_matrix(y_test,y_predict)
score=accuracy_score(y_test,y_predict)
run.log('accuracy',score)
import joblib
model_file='./outputs/model.pkl'
joblib.dump(value=[train_encoded_cols,trained_model],filename=model_file)

run.complete()


In [30]:
#register the model
new_run=ws.get_run('69ed75c0-7cea-49c3-aa10-d99bd985aca2')
new_run.register_model(model_name='adultincome',model_path='outputs/model.pkl')

Model(workspace=Workspace.create(name='hyperdive12', subscription_id='1db33695-8135-4616-9bb4-9574b401d454', resource_group='defaultsrg02'), name=adultincome, id=adultincome:1, version=1, tags={}, properties={})

In [1]:
Model.list(ws)

NameError: name 'Model' is not defined

In [36]:
d1={'a':10,'b':20,'c':30}
import json
json_d1=json.dumps(d1)
json.loads(json_d1)

{'a': 10, 'b': 20, 'c': 30}

In [ ]:
#create aks cluster
cluster_name='akscluster'
aksconfig=AksCompute.provisioning_configuration(location='eastus',vm_size='STANDARD_D11_V2,'agenet_count=1,cluster_purpose='DevTest')
akscluster=ComputeTarget.create(ws,cluster_name,aksconfig)

#create inference config
inference_config=InferenceConfig(entry_script='entry script.py',environment=exp)
#deply config
deployconfig=AksWebservice.deploy_configuration(cpu_cores=1,memory_gb=1,enable_app_insights=True)

#deploy the webservice
from azureml.core.model import Model
model=ws.models['adultincome']
service=Model.deploy(workspace=ws,name='adultincomeservice',models=[model],inference_config=inference_config,deployment_config=deployconfig,deployment_targwet=akscluster)
service.wait_for_deployment(show_output=True)
